# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [1]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [16]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'


#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions). Hint: use the headers parameter to get the data!

In [17]:
# headers definition

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
response = requests.get(url, headers=headers)
results = response.json()
results


{'total_count': 22526,
 'pagination': {'page_count': 269, 'current_page': 4, 'per_page': 84},
 'sort': 'sale',
 'articles': [{'sku': 'NA823L0E1-I11',
   'name': "NKFMOVE - Veste d'hiver - cerise",
   'price': {'original': '29,99\xa0€',
    'promotional': '11,99\xa0€',
    'has_different_prices': False,
    'has_different_original_prices': False,
    'has_different_promotional_prices': False,
    'has_discount_on_selected_sizes_only': False},
   'sizes': ['8a', '10a', '11a', '12a', '13a'],
   'url_key': 'name-it-nkfmove-jacket-veste-dhiver-cerise-na823l0e1-i11',
   'media': [{'path': 'NA/82/3L/0E/1I/11/NA823L0E1-I11@8.jpg',
     'role': 'DEFAULT',
     'packet_shot': False}],
   'brand_name': 'Name it',
   'is_premium': False,
   'family_articles': [{'sku': 'NA823L0E1-I11',
     'url_key': 'name-it-nkfmove-jacket-veste-dhiver-cerise-na823l0e1-i11',
     'media': [{'path': 'NA/82/3L/0E/1I/11/NA823L0E1-I11@8.jpg',
       'role': 'FAMILY',
       'packet_shot': False}],
     'name': "NKFMO

In [18]:
# Your code
flattened_data = json_normalize(results)
flattened_data

flattened_data2= json_normalize(flattened_data["articles"][0]) #normlaizing first element because have only one item
flattened_data2

,total_count,sort,articles,query_path,previous_page_path,next_page_path,page_gender,premium,filters,total_article_count,...,iconPaths.filters.standard_delivery_filter,iconPaths.filters.fast_delivery_filter,iconPaths.filters.zalando_plus,iconPaths.mobileFilters.standard_delivery_filter,iconPaths.mobileFilters.fast_delivery_filter,iconPaths.mobileFilters.zalando_plus,iconPaths.flags.slow_delivery_flag,iconPaths.flags.fast_delivery_flag,iconPaths.flags.plus_delivery_flag,iconPaths.flags.zalando_plus
0,22526,sale,"[{'sku': 'NA823L0E1-I11', 'name': 'NKFMOVE - V...",/promo-enfant/?p=4&order=sale,/promo-enfant/?p=3&order=sale,/promo-enfant/?p=5&order=sale,kids,False,"[{'key': 'sizes', 'label': 'Taille', 'url_key'...",22520,...,icons/truck.svg,icons/truck-fast.svg,icons/plus-short-1.svg,icons/truck.svg,icons/truck-fast.svg,icons/plus-short-1.svg,icons/clock.svg,icons/truck-fast-orange-3.svg,icons/plus-short-1.svg,icons/zalando-plus.svg


,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only
0,NA823L0E1-I11,NKFMOVE - Veste d'hiver - cerise,"[8a, 10a, 11a, 12a, 13a]",name-it-nkfmove-jacket-veste-dhiver-cerise-na8...,[{'path': 'NA/82/3L/0E/1I/11/NA823L0E1-I11@8.j...,Name it,False,"[{'sku': 'NA823L0E1-I11', 'url_key': 'name-it-...","[{'key': 'discountRate', 'value': '-60%', 'tra...",clothing,[],"29,99 €","11,99 €",False,False,False,False
1,NI114A03I-A00,AIR FORCE 1 - Baskets montantes - white,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40]",nike-sportswear-air-force-1-mid-06-baskets-mon...,[{'path': 'NI/11/4A/03/IA/00/NI114A03I-A00@19....,Nike Sportswear,False,"[{'sku': 'NI114A03I-A00', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -20...",shoe,[],"79,95 €","63,95 €",True,False,True,False
2,F5723L00Q-Q11,Veste d'hiver - black iris,"[5-6a, 7-8a, 9-10a, 11-12a, 13-14a, 15-16a]",friboo-veste-dhiver-black-iris-f5723l00q-q11,[{'path': 'F5/72/3L/00/QQ/11/F5723L00Q-Q11@12....,Friboo,False,"[{'sku': 'F5723L00Q-Q11', 'url_key': 'friboo-v...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"42,95 €","34,35 €",False,False,False,False
3,NA823J0EH-J11,NMFNAILA CARD - Gilet - strawberry cream,"[2a, 3a, 4a, 5a, 6a]",name-it-nmfnaila-card-gilet-strawberry-cream-n...,[{'path': 'NA/82/3J/0E/HJ/11/NA823J0EH-J11@7.j...,Name it,False,"[{'sku': 'NA823J0EH-J11', 'url_key': 'name-it-...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"22,99 €","18,39 €",False,False,False,False
4,R2724G00N-G11,EZRA - T-shirt à manches longues - bordeaux,"[12a, 14a, 16a]",redskins-ezra-t-shirt-a-manches-longues-r2724g...,[{'path': 'R2/72/4G/00/NG/11/R2724G00N-G11@7.j...,Redskins,False,"[{'sku': 'R2724G00N-G11', 'url_key': 'redskins...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"23,95 €","19,15 €",False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,TO126K008-A11,UNISEX FLAG - Sweatshirt - classic white,"[4a, 6a, 8a, 10a, 12a, 18-24m]",tommy-hilfiger-unisex-flag-sweatshirt-classic-...,[{'path': 'TO/12/6K/00/8A/11/TO126K008-A11@9.j...,Tommy Hilfiger,False,"[{'sku': 'TO126K008-A11', 'url_key': 'tommy-hi...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"84,95 €","67,95 €",True,False,True,False
80,ES123F0A8-K11,DRESS - Robe d'été - navy,"[10-11a, 12-13a, 14y, 15-16a]",esprit-dress-robe-dete-navy-es123f0a8-k11,[{'path': 'ES/12/3F/0A/8K/11/ES123F0A8-K11@6.j...,Esprit,False,"[{'sku': 'ES123F0A8-K11', 'url_key': 'esprit-d...","[{'key': 'discountRate', 'value': 'Jusqu’à -40...",clothing,[],"39,99 €","23,99 €",True,False,True,False
81,PU143E0B6-Q11,ACTIVE SPORTS PANTS - Pantalon de survêtement...,"[4a, 8a, 10a, 12a, 14a, 16a]",puma-active-sports-pants-pantalon-de-surveteme...,[{'path': 'PU/14/3E/0B/6Q/11/PU143E0B6-Q11@5.j...,Puma,False,"[{'sku': 'PU143E0B6-Q11', 'url_key': 'puma-act...","[{'key': 'discountRate', 'value': '-25%', 'tra...",clothing,[],"34,95 €","26,15 €",False,False,False,False
82,NI126G00U-Q11,AIR - T-shirt à manches longues - black/dark s...,"[8-9a, 10-11a, 12-13a, 14a]",nike-sportswear-air-t-shirt-a-manches-longues-...,[{'path': 'NI/12/6G/00/UQ/11/NI126G00U-Q11@6.j...,Nike Sportswear,False,"[{'sku': 'NI126G00U-Q11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-35%', 'tra...",clothing,[],"29,95 €","19,45 €",False,False,False,False


#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [22]:
# Get the total number of pages

# Your code
total_pages = results["pagination"]["page_count"] #go inside of pagination and take page_count
total_pages

total_items = results["total_count"]
total_items


269

22532

In [66]:
#if I pass this to the range, do I need to add 1 so that it would take the full number??
total_items = results["total_count"]+1
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

parameters = {"sort":"sale"}

df_list = []

for offset in range(0, total_items, 84): #168 is items, later to change to total_items
    url = "https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset="+str(offset)              
    response = requests.get(url=url, headers=headers, params=parameters)        
    results = response.json()
    flattened_data = json_normalize(results)
    data = json_normalize(flattened_data["articles"][0]) #normlaizing first element because have only one item
    df_list.append(data)

all_dfs = pd.concat(df_list)
all_dfs


C:\Users\krist\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


,amount,brand_name,delivery_promises,family_articles,flags,is_premium,media,name,price.base_price,price.has_different_original_prices,...,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key
0,NaN,Tommy Hilfiger,[],"[{'sku': 'TO113I00A-Q11', 'url_key': 'tommy-hi...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'TO/11/3I/00/AQ/11/TO113I00A-Q11@10....,BOOT - Bottines à lacets - black,NaN,False,...,False,"99,95 €","79,95 €",shoe,"[28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 3...",TO113I00A-Q11,https://ccp-et.metrigo.zalan.do/event/sbv?z=1f...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=1...,ccp,tommy-hilfiger-boot-bottines-a-lacets-black-to...
1,NaN,Tommy Hilfiger,[],"[{'sku': 'TO114D01J-M11', 'url_key': 'tommy-hi...","[{'key': 'discountRate', 'value': 'Jusqu’à -35...",False,[{'path': 'TO/11/4D/01/JM/11/TO114D01J-M11@11....,Baskets basses - green,NaN,False,...,False,"59,95 €","41,95 €",shoe,"[24, 25, 26, 27, 28, 29, 30, 32, 33]",TO114D01J-M11,https://ccp-et.metrigo.zalan.do/event/sbv?z=1f...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=1...,ccp,tommy-hilfiger-baskets-basses-green-to114d01j-m11
2,NaN,Tommy Hilfiger,[],"[{'sku': 'TO116D00C-K11', 'url_key': 'tommy-hi...","[{'key': 'discountRate', 'value': '-10%', 'tra...",False,[{'path': 'TO/11/6D/00/CK/11/TO116D00C-K11@11....,Baskets basses - blue,NaN,False,...,False,"64,95 €","58,45 €",shoe,"[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]",TO116D00C-K11,https://ccp-et.metrigo.zalan.do/event/sbv?z=1f...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=1...,ccp,tommy-hilfiger-baskets-basses-blue-to116d00c-k11
3,NaN,Levi's®,[],"[{'sku': 'LE223K01U-C11', 'url_key': 'levisr-k...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'LE/22/3K/01/UC/11/LE223K01U-C11@6.j...,KEY ITEM LOGO CREW - Sweatshirt - grey heather,NaN,False,...,False,"38,95 €","31,15 €",clothing,"[4a, 8a, 10a, 12a, 14a, 16a]",LE223K01U-C11,NaN,NaN,NaN,levisr-key-item-logo-crew-sweatshirt-le223k01u...
4,NaN,adidas Originals,[],"[{'sku': 'AD126L00J-Q11', 'url_key': 'adidas-o...","[{'key': 'discountRate', 'value': 'Jusqu’à -35...",False,[{'path': 'AD/12/6L/00/JQ/11/AD126L00J-Q11@8.j...,JACKET - Veste d'hiver - black/white,NaN,False,...,False,"64,95 €","41,95 €",clothing,"[3-4a, 4-5a, 5-6a, 6-7a, 7-8a]",AD126L00J-Q11,NaN,NaN,NaN,adidas-originals-jacket-veste-dhiver-blackwhit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,NaN,Nike Performance,[],"[{'sku': 'N1243D12J-Q11', 'url_key': 'nike-per...",[],False,[{'path': 'N1/24/3D/12/JQ/11/N1243D12J-Q11@5.j...,INTER MAILAND DRY - Article de supporter - black,NaN,False,...,False,"55,00 €","55,00 €",clothing,[8-10a],N1243D12J-Q11,NaN,NaN,NaN,nike-performance-inter-mailand-dry-t-shirt-a-m...
79,NaN,Superfit,[],"[{'sku': 'LE314E01O-K11', 'url_key': 'superfit...","[{'key': 'discountRate', 'value': 'Jusqu’à -20...",False,[{'path': 'LE/31/4E/01/OK/11/LE314E01O-K11@3.j...,EARTH - Baskets basses - blau/hellblau,NaN,True,...,False,"54,95 €","54,95 €",shoe,"[34, 38]",LE314E01O-K11,NaN,NaN,NaN,superfit-earth-baskets-basses-le314e01o-k11
80,NaN,Ricosta,[],"[{'sku': 'RI213G03C-D11', 'url_key': 'ricosta-...","[{'key': 'discountRate', 'value': 'Jusqu’à -20...",False,[{'path': 'RI/21/3G/03/CD/11/RI213G03C-D11@8.j...,CHICA - Sandales - silber,NaN,True,...,False,"41,95 €","41,95 €",shoe,"[26, 29, 34]",RI213G03C-D11,NaN,NaN,NaN,ricosta-chica-sandales-silber-ri213g03c-d11
81,NaN,Primigi,[],"[{'sku': 'PR314F00L-K11', 'url_key': 'primigi-...","[{'key': 'discountRate', 'value': 'Jusqu’à -22...",False,[{'path': 'PR/31/4F/00/LK/11/PR314F00L-K11@3.j...,Baskets montantes - jeans,NaN,True,...,False,"39,95 €","39,95 €",shoe,"[22, 25]",PR314F00L-K11,NaN,NaN,NaN,primigi-sandales-jeans-pr314f00l-k11


#### Display the trending brand in DataFrame

In [71]:
# your code
all_dfs['brand_name'].value_counts()

#the answer is Name it

Name it            879
GAP                720
Boboli             675
Esprit             664
Friboo             612
                  ... 
le coq sportif       1
Deuter               1
18CRR81 Cerruti      1
Weleda               1
Cat Footwear         1
Name: brand_name, Length: 348, dtype: int64

#### Display the brand with maximal total discount (sum of discounts on all goods)

In [88]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [124]:
#Our data is still text. Convert prices into numbers.
# your code


all_dfs['original price'] = all_dfs['price.original'].str.extract(r'(\d{2}\D\d{2})').replace('\D', '.', regex=True)
all_dfs['promotional price'] = all_dfs['price.promotional'].str.extract(r'(\d{2}\D\d{2})').replace('\D', '.', regex=True)

all_dfs['original price'] = pd.to_numeric(all_dfs['original price'])
all_dfs['promotional price'] = pd.to_numeric(all_dfs['promotional price'])


In [162]:
all_dfs['discount'] = all_dfs['original price'] - all_dfs['promotional price']
all_dfs.head()

,amount,brand_name,delivery_promises,family_articles,flags,is_premium,media,name,price.base_price,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key,original price,promotional price,discount
0,NaN,Tommy Hilfiger,[],"[{'sku': 'TO113I00A-Q11', 'url_key': 'tommy-hilfiger-boot-bottines-a-lacets-black-to113i00a-q11', 'media': [{'path': 'TO/11/3I/00/AQ/11/TO113I00A-Q11@11.jpg', 'role': 'FAMILY', 'packet_shot': True}], 'name': 'BOOT - Bottines à lacets - black', 'sizes': ['28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40'], 'price': {'original': '99,95 €', 'promotional': '79,95 €', 'has_different_prices': False, 'has_different_original_prices': False, 'has_different_promotional_prices': False, 'has_discount_on_selected_sizes_only': False}, 'flags': [{'key': 'discountRate', 'value': '-20%', 'tracking_value': 'discount rate'}], 'delivery_promises': []}]","[{'key': 'discountRate', 'value': '-20%', 'tracking_value': 'discount rate'}, {'key': 'sponsored', 'value': 'Sponsorisé', 'tracking_value': 'sponsored'}]",False,"[{'path': 'TO/11/3I/00/AQ/11/TO113I00A-Q11@10.jpg', 'role': 'DEFAULT', 'packet_shot': False}]",BOOT - Bottines à lacets - black,NaN,False,False,False,False,"99,95 €","79,95 €",shoe,"[28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]",TO113I00A-Q11,https://ccp-et.metrigo.zalan.do/event/sbv?z=1fb779a9-7739-4ae4-b3b1-e8b8fe745ffd&b=8978b507-4c90-44f3-9c47-37256c9745c7:30032878:TO113I00A-Q11&c=30032878&a=30032882&s=TO113I00A-Q11&is=1689&at=browser&cm=140000&ovm=500&r=0.004488551057875156&ts=0&cs=4208668920,[https://ccp-et.metrigo.zalan.do/event/sbv?z=1fb779a9-7739-4ae4-b3b1-e8b8fe745ffd&b=8978b507-4c90-44f3-9c47-37256c9745c7:30032878:TO113I00A-Q11&c=30032878&a=30032882&s=TO113I00A-Q11&is=1689&at=browser&cm=140000&ovm=500&r=0.004488551057875156&ts=0&cs=4208668920],ccp,tommy-hilfiger-boot-bottines-a-lacets-black-to113i00a-q11,99.95,79.95,20.0
1,NaN,Tommy Hilfiger,[],"[{'sku': 'TO114D01J-M11', 'url_key': 'tommy-hilfiger-baskets-basses-green-to114d01j-m11', 'media': [{'path': 'TO/11/4D/01/JM/11/TO114D01J-M11@11.jpg', 'role': 'FAMILY', 'packet_shot': True}], 'name': 'Baskets basses - green', 'sizes': ['24', '25', '26', '27', '28', '29', '30', '32', '33'], 'price': {'original': '59,95 €', 'promotional': '41,95 €', 'has_different_prices': False, 'has_different_original_prices': False, 'has_different_promotional_prices': False, 'has_discount_on_selected_sizes_only': False}, 'flags': [{'key': 'discountRate', 'value': 'Jusqu’à -35%', 'tracking_value': 'discount rate'}], 'delivery_promises': []}, {'sku': 'TO114D01J-C11', 'url_key': 'tommy-hilfiger-baskets-basses-greyblue-to114d01j-c11', 'media': [{'path': 'TO/11/4D/01/JC/11/TO114D01J-C11@10.jpg', 'role': 'FAMILY', 'packet_shot': True}], 'name': 'Baskets basses - grey/blue', 'sizes': ['28'], 'price': {'original': '59,95 €', 'promotional': '47,95 €', 'has_different_prices': False, 'has_different_original_prices': False, 'has_different_promotional_prices': False, 'has_discount_on_selected_sizes_only': False}, 'flags': [{'key': 'discountRate', 'value': '-20%', 'tracking_value': 'discount rate'}], 'delivery_promises': []}]","[{'key': 'discountRate', 'value': 'Jusqu’à -35%', 'tracking_value': 'discount rate'}, {'key': 'sponsored', 'value': 'Sponsorisé', 'tracking_value': 'sponsored'}]",False,"[{'path': 'TO/11/4D/01/JM/11/TO114D01J-M11@11.jpg', 'role': 'DEFAULT', 'packet_shot': True}]",Baskets basses - green,NaN,False,False,False,False,"59,95 €","41,95 €",shoe,"[24, 25, 26, 27, 28, 29, 30, 32, 33]",TO114D01J-M11,https://ccp-et.metrigo.zalan.do/event/sbv?z=1fb779a9-7739-4ae4-b3b1-e8b8fe745ffd&b=8978b507-4c90-44f3-9c47-37256c9745c7:30032878:TO114D01J-M11&c=30032878&a=30032882&s=TO114D01J-M11&is=1689&at=brow

In [129]:
max_disc = all_dfs.groupby('brand_name', as_index=False)['discount'].sum().sort_values('discount',ascending=False).head(10)
max_disc

,brand_name,discount
25,Boboli,8188.30
83,Esprit,6426.03
91,Friboo,5320.85
332,adidas Performance,4652.96
196,Name it,4571.25
301,Tommy Hilfiger,4293.65
204,Nike Performance,4007.03
233,Puma,3863.38
96,GAP,3691.05
206,Nike Sportswear,3440.82


In [160]:
max_disc.loc[max_disc['discount'].idxmax()]

brand_name    Boboli
discount      8188.3
Name: 25, dtype: object

#### Display the brands without discount at all

In [149]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [161]:
# your code

max_disc[max_disc['discount'] == 0]

#don't have brands without total 0 discount

,brand_name,discount
